The following python notebook is a summary of the first half of EECS127 - Optimization Models in Engineering. Every type of regression can be generalized as a special case of Tikhonov regression, and thus we build a class that allows for the construction of different regression types.

## Tikhonov Regression

Let 
$ A \in \mathbb{R}^{m \times n} $, 
$ \mathbf{\tilde{x}_0} \in \mathbb{R}^n $,
$ \mathbf{\tilde{y}} \in \mathbb{R}^m $, 
and let 
$ W_1 \in \mathbb{R}^{m \times m} $ 
and 
$ W_2 \in \mathbb{R}^{n \times n} $ 
be diagonal. Then the unique solution to the Tikhonov regression problem is:

$$
\min_{\mathbf{x} \in \mathbb{R}^n} \left\{ \| W_1 (A\mathbf{x} - \mathbf{\tilde{y}})\|_2^2 + \| W_2 (\mathbf{x} - \mathbf{\tilde{x}_0})\|_2^2 \right\} \quad (4.62)
$$

The solution is given by:

$$
\mathbf{x^*} = (A^T W_2^2 A + W_2^2)^{-1} (A^T W_2^2 \mathbf{\tilde{y}} + W_2^2 \mathbf{\tilde{x}_0}) \quad (4.63)
$$


In [2]:
import numpy as np

class TikhonovRegression:
    def __init__(self, W1=None, W2=None):
        """
        Initialize the TikhonovRegression class.
        
        Parameters:
        - W1: Diagonal matrix of shape (m, m)
        - W2: Diagonal matrix of shape (n, n)
        """
        self.W1 = W1
        self.W2 = W2
        self.x_star = None

    def fit(self, A, y_tilde, x0_tilde):
        """
        Fit the Tikhonov regression model.
        
        Parameters:
        - A: Matrix of shape (m, n)
        - y_tilde: Vector of shape (m,)
        - x0_tilde: Vector of shape (n,)
        """
        if self.W1 is None:
            self.W1 = np.eye(A.shape[0])
        
        if self.W2 is None:
            self.W2 = np.eye(A.shape[1])

        # Compute the solution using the provided equation
        inv_term = np.linalg.inv(A.T @ np.linalg.matrix_power(self.W1, 2) @ A + np.linalg.matrix_power(self.W2, 2))
        self.x_star = inv_term @ (A.T @ np.linalg.matrix_power(self.W1, 2) @ y_tilde + np.linalg.matrix_power(self.W2, 2) @ x0_tilde)

    def predict(self, A):
        """
        Predict using the Tikhonov regression model.
        
        Parameters:
        - A: Matrix of shape (m, n)
        
        Returns:
        Predicted vector of shape (m,)
        """
        if self.x_star is None:
            raise ValueError("Model has not been fit yet!")

        return A @ self.x_star


In [3]:
class RidgeRegression(TikhonovRegression):
    def __init__(self, lambda_):
        """
        Initialize the RidgeRegression class.
        
        Parameters:
        - lambda_: Regularization parameter
        """
        # W1 is identity by default in TikhonovRegression, so we don't need to specify it here
        W2 = np.sqrt(lambda_) * np.eye(x0_tilde.shape[0])  # W2^2 will then be lambda * identity
        super().__init__(W2=W2)
    
    def fit(self, A, y_tilde):
        """
        Fit the Ridge regression model.
        
        Parameters:
        - A: Matrix of shape (m, n)
        - y_tilde: Vector of shape (m,)
        """
        # For Ridge Regression, x0_tilde is the zero vector
        x0_tilde = np.zeros(A.shape[1])
        
        # Call the fit method of the parent TikhonovRegression class
        super().fit(A, y_tilde, x0_tilde)

In [4]:
class LeastSquaresRegression(TikhonovRegression):
    def __init__(self):
        """
        Initialize the LeastSquaresRegression class.
        No need for W2 in this case since the regularization term is dropped.
        """
        super().__init__()

    def fit(self, A, y_tilde):
        """
        Fit the Least Squares regression model.
        
        Parameters:
        - A: Matrix of shape (m, n)
        - y_tilde: Vector of shape (m,)
        """
        # For Least Squares, x0_tilde is the zero vector
        x0_tilde = np.zeros(A.shape[1])
        
        # Overriding the fit method to drop the regularization term
        self.x_star = np.linalg.inv(A.T @ A) @ A.T @ y_tilde